In [ ]:
!pip install evaluate
!pip install bert-score
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

4.51.1


In [ ]:
model_checkpoint = 'vinai/phobert-base-v2'

In [ ]:
from transformers import AutoTokenizer
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
tokenizer_train = AutoTokenizer.from_pretrained('/content/drive/MyDrive/data/tokenizer/train/')
tokenizer_val = AutoTokenizer.from_pretrained('/content/drive/MyDrive/data/tokenizer/valid/')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/data/tokenizer/train/vocab.txt', sep=' ', header=None, names=['token', 'id'])
df.sample(5)

,token,id
28126,cúm_@@,671
12654,Ép,2555
10411,Cẩm_Nhung,2776
20693,BTNMT,1992
20957,ngke,1977


In [ ]:
from transformers import EncoderDecoderModel

# Đặt tying = True
shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base-v2", "vinai/phobert-base-v2", tie_encoder_decoder=True)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.1.crossattention.output.dense.weight', 'roberta.en

In [ ]:
# Token mở đầu, kết thúc, padding
shared.config.decoder_start_token_id = tokenizer.bos_token_id
shared.config.eos_token_id = tokenizer.eos_token_id
shared.config.pad_token_id = tokenizer.eos_token_id

# Parameter cho decoding
shared.config.max_length = 64
shared.config.early_stopping = True
shared.config.no_repeat_ngram_size = 3
shared.config.length_penalty = 2.0
shared.config.num_beams = 4
shared.config.vocab_size = shared.config.encoder.vocab_size

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/seq2seq')

from transformers import TrainingArguments
from transformers import Seq2SeqTrainer

from dataclasses import dataclass, field
from typing import Optional
from datasets import Dataset

In [ ]:
# Thêm generation_config vào Seq2SeqTrainingArguments
# Copy class Seq2SeqTrainingArguments trong file seq2seq_trainer.py
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0
    )
    sortish_sampler: bool = field(default=False)
    predict_with_generate: bool = field(
        default=False
    )
    adafactor: bool = field(default=False)
    encoder_layerdrop: Optional[float] = field(
        default=None
    )
    decoder_layerdrop: Optional[float] = field(
        default=None
    )
    dropout: Optional[float] = field(default=None)
    attention_dropout: Optional[float] = field(
        default=None
    )
    lr_scheduler: Optional[str] = field(
        default="linear"
    )
    generation_config: Optional[str] = field(
        default=None
    )

In [ ]:
from evaluate import load
import numpy as np
from bert_score import score

def compute_metrics(pred):
    # Load metrics
    rouge = load("rouge")
    bertscore = load("bertscore")

    # Decode predictions và labels
    pred_ids = pred.predictions
    labels_ids = pred.label_ids

    # Decode sang text
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Tính ROUGE scores cho tất cả loại
    rouge_results = rouge.compute(
        predictions=pred_str,
        references=label_str,
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
        use_aggregator=True  # Tính giá trị trung bình
    )

    # Tính BERTScore (sử dụng model phù hợp cho tiếng Việt)
    bertscore_results = bertscore.compute(
        predictions=pred_str,
        references=label_str,
        model_type=model_checkpoint,  # Hoặc model_checkpoint nếu phù hợp
        lang="vi",
        device="cuda" if torch.cuda.is_available() else "cpu"  # Tăng tốc độ tính toán
    )

    return {
        # ROUGE-1
        "rouge1_precision": round(rouge_results["rouge1"].mid.precision, 4),
        "rouge1_recall": round(rouge_results["rouge1"].mid.recall, 4),
        "rouge1_f1": round(rouge_results["rouge1"].mid.fmeasure, 4),

        # ROUGE-2
        "rouge2_precision": round(rouge_results["rouge2"].mid.precision, 4),
        "rouge2_recall": round(rouge_results["rouge2"].mid.recall, 4),
        "rouge2_f1": round(rouge_results["rouge2"].mid.fmeasure, 4),

        # ROUGE-L
        "rougeL_precision": round(rouge_results["rougeL"].mid.precision, 4),
        "rougeL_recall": round(rouge_results["rougeL"].mid.recall, 4),
        "rougeL_f1": round(rouge_results["rougeL"].mid.fmeasure, 4),

        # ROUGE-Lsum
        "rougeLsum_precision": round(rouge_results["rougeLsum"].mid.precision, 4),
        "rougeLsum_recall": round(rouge_results["rougeLsum"].mid.recall, 4),
        "rougeLsum_f1": round(rouge_results["rougeLsum"].mid.fmeasure, 4),

        # BERTScore (trung bình các mẫu)
        "bertscore_precision": round(np.mean(bertscore_results["precision"]), 4),
        "bertscore_recall": round(np.mean(bertscore_results["recall"]), 4),
        "bertscore_f1": round(np.mean(bertscore_results["f1"]), 4),
    }

In [ ]:
import os

# Tắt wandb
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Hyperparameters
batch_size = 16
encoder_max_length = 256
decoder_max_length = 100

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/data/splitting/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/data/splitting/val.csv')

train_df.head(), val_df.head()

(                                                 URL  \
 0  https://thanhnien.vn/tuyen-sinh-lop-10-tp-hcm-...   
 1  https://thanhnien.vn/nph-lien-minh-huyen-thoai...   
 2  https://tuoitre.vn/danh-giac-corona-theo-phong...   
 3  https://thanhnien.vn/toyota-gioi-thieu-he-thon...   
 4  https://tuoitre.vn/covid-19-khien-ti-le-thai-l...   
 
                                                Title  \
 0  tuyển_sinh lớp 10 tphcm đề thi_môn tiếng anh k...   
 1  nph liên_minh huyền_thoại trêu game_thủ bằng t...   
 2           đánh giặc corona theo phong_cách ngành y   
 3  toyota giới_thiệu hệ_thống sạc điện_thoại khôn...   
 4  covid19 khiến tỉ_lệ thai lưu_sản_phụ tử_vong t...   
 
                                             Abstract  \
 0  chiều 116 gần 94000 học_sinh lớp 9 ở tphcm đã ...   
 1  sau liên_minh huyền_thoại fifa online 3 và chi...   
 2  tto phòng_khám đa_khoa trường đại_học y_khoa p...   
 3  các hệ_thống sạc pin điện_thoại không dây đã b...   
 4  tto theo một nghiên_cứu

In [ ]:
# Hợp Content and Abstract thành input text
train_df["input_text"] = train_df['Category'].fillna("") + " " + train_df["Abstract"].fillna("") + " " + train_df["Content"].fillna("")
val_df["input_text"] = val_df['Category'].fillna("") + " " + val_df["Abstract"].fillna("") + " " + val_df["Content"].fillna("")

# Dropna
train_df = train_df.dropna(subset=["Title"])
val_df = val_df.dropna(subset=["Title"])

# Drop URL, Category
train_df = train_df.drop(columns=['URL'])
val_df = val_df.drop(columns=['URL'])

# Chuyển sang HuggingFace dataset
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

In [ ]:
train_data

Dataset({
    features: ['Title', 'Abstract', 'Content', 'Category', 'input_text'],
    num_rows: 48000
})

In [ ]:
val_data

Dataset({
    features: ['Title', 'Abstract', 'Content', 'Category', 'input_text'],
    num_rows: 6000
})

In [ ]:
# Process data
def process_data_to_model_inputs(batch, tokenizer):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Title"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs["input_ids"]
    batch["attention_mask"] = inputs["attention_mask"]
    batch["labels"] = outputs["input_ids"].copy()
    # Mask loss cho padding
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
    # assert len(batch["input_ids"][0]) == encoder_max_length, "Input IDs length mismatch!"
    # assert len(batch["Title"][0]) == decoder_max_length, "Decoder Input IDs length mismatch!"
    # assert len(batch["attention_mask"][0]) == encoder_max_length, "Attention Mask length mismatch!"
    # assert len(batch["decoder_attention_mask"][0]) == decoder_max_length, "Decoder Attention Mask length mismatch!"
    return batch

In [ ]:
# Process training data
train_data_batch = train_data.map(
    lambda batch: process_data_to_model_inputs(batch, tokenizer_train),
    batched=True,
    batch_size=batch_size,
    remove_columns=["Title", "Abstract", "Content", "Category"],  # Bỏ cột cũ
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

# Process validation data
val_data_batch = val_data.map(
    lambda batch: process_data_to_model_inputs(batch, tokenizer_val),
    batched=True,
    batch_size=batch_size,
    remove_columns=["Title", "Abstract", "Content", "Category"],  # Bỏ cột cũ
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/training/',
    per_device_train_batch_size=batch_size,  # Tăng nếu GPU mạnh
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=500,  # Ghi log sau mỗi 500 bước
    save_steps=5000,  # Lưu checkpoint sau mỗi 5000 bước
    eval_steps=2500,  # Đánh giá sau mỗi 2500 bước
    warmup_steps=2500,  # Warm-up trong 2500 bước đầu tiên
    num_train_epochs=5,
    learning_rate=2e-5,  # Learning rate mặc định
    overwrite_output_dir=True,
    save_total_limit=10,  # Giới hạn lưu 10 checkpoint
    fp16=True,  # Sử dụng mixed precision
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
trainer = Seq2SeqTrainer(
    model=shared,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:631: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss
500,9.235300
1000,5.913700
1500,5.067800
2000,4.715200
2500,4.441600
3000,4.269900
3500,3.907800
4000,3.792500
4500,3.706200
5000,3.632900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:631: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:651: FutureWarning: Version v4.12.0 introduces a better way to t

TrainOutput(global_step=15000, training_loss=3.5132443033854166, metrics={'train_runtime': 5986.1831, 'train_samples_per_second': 40.092, 'train_steps_per_second': 2.506, 'total_flos': 4.227178991616e+16, 'train_loss': 3.5132443033854166, 'epoch': 5.0})

In [ ]:
# Lấy state của trainer
training_stats = trainer.state

print("🔥🔥🔥:")
print(training_stats)

🔥🔥🔥:
TrainerState(epoch=5.0, global_step=15000, max_steps=15000, logging_steps=500, eval_steps=2500, save_steps=5000, train_batch_size=16, num_train_epochs=5, num_input_tokens_seen=0, total_flos=4.227178991616e+16, log_history=[{'loss': 9.2353, 'grad_norm': 26.0057430267334, 'learning_rate': 3.920000000000001e-06, 'epoch': 0.16666666666666666, 'step': 500}, {'loss': 5.9137, 'grad_norm': 11.588826179504395, 'learning_rate': 7.92e-06, 'epoch': 0.3333333333333333, 'step': 1000}, {'loss': 5.0678, 'grad_norm': 11.518630981445312, 'learning_rate': 1.1920000000000001e-05, 'epoch': 0.5, 'step': 1500}, {'loss': 4.7152, 'grad_norm': 11.879873275756836, 'learning_rate': 1.5920000000000003e-05, 'epoch': 0.6666666666666666, 'step': 2000}, {'loss': 4.4416, 'grad_norm': 10.962403297424316, 'learning_rate': 1.9920000000000002e-05, 'epoch': 0.8333333333333334, 'step': 2500}, {'loss': 4.2699, 'grad_norm': 10.473255157470703, 'learning_rate': 1.9216e-05, 'epoch': 1.0, 'step': 3000}, {'loss': 3.9078, 'gra

In [ ]:
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer

output_dir = '/content/drive/MyDrive/training/'
tokenizer_test = AutoTokenizer.from_pretrained('/content/drive/MyDrive/data/tokenizer/test/', use_fast=False)
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/training/checkpoint-15000", use_fast=False)
model = EncoderDecoderModel.from_pretrained(output_dir + "/checkpoint-15000")
model.to("cuda")

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [ ]:
!pip install transformers datasets evaluate bert-score

In [ ]:
import pandas as pd
# Import dataset from huggingface
from datasets import Dataset
from evaluate import load
import numpy as np
from bert_score import score
import evaluate

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/data/splitting/test.csv')

# Hợp Content and Abstract thành input text
test_df["input_text"] = test_df['Category'].fillna("") + " " + test_df["Abstract"].fillna("") + " " + test_df["Content"].fillna("")

# Dropna
test_df = test_df.dropna(subset=["Title"])

# Drop URL, Category
test_df = test_df.drop(columns=['URL'])

# Chuyển sang HuggingFace dataset
test_data = Dataset.from_pandas(test_df)

test_data

Dataset({
    features: ['Title', 'Abstract', 'Content', 'Category', 'input_text'],
    num_rows: 6000
})

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e1c5f88a910ad3213999b66cab09cc4c70cdc0e753f39810c0e963f23cd4c958
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [39]:
# Logic gần giống việc map train ở trên
def title_generator(batch):
    category = batch["Category"]
    abstract = batch["Abstract"]
    content = batch["Content"]

    inputs_text = [f"{c} {a} {ct}" for c, a, ct in zip(category, abstract, content)]

    inputs = tokenizer_test(inputs_text,
                       padding="max_length",
                       truncation=True,
                       max_length=256,
                       return_tensors="pt")

    inputs = {key: value.to("cuda") for key, value in inputs.items()}

    outputs = model.generate(
        inputs["input_ids"],
        max_length=80,
        num_beams=4,
        early_stopping=True
    )
    # print(outputs)
    batch["predicted_title"] = tokenizer_test.batch_decode(outputs, skip_special_tokens=True)
    # print(len(batch['Title']))
    # print(len(batch["predicted_title"]))
    return batch

In [ ]:
batch_size = 16
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# test_data_batch = test_data.map(
#     lambda batch: process_data_to_model_inputs(batch, tokenizer_test),
#     batched=True,
#     batch_size=batch_size,
#     remove_columns=["Title", "Abstract", "Content", "Category"],  # Bỏ cột cũ
# )
# test_data_batch.set_format(
#     type="torch", columns=["input_ids", "attention_mask", "labels"],
# )

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [40]:
test_result = test_data.map(
    lambda batch: title_generator(batch),
    batched=True,
    batch_size=batch_size,
    remove_columns=["Title", "Abstract", "Content", "Category"]
)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [41]:
# Lấy tiêu đề thực tế và tiêu đề dự đoán
references = test_df["Title"]
predictions = test_result["predicted_title"]

In [42]:
# Tính ROUGE
rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_scores['rouge1']}")
print(f"ROUGE-2: {rouge_scores['rouge2']}")
print(f"ROUGE-L: {rouge_scores['rougeL']}")
print(f"ROUGE-LSUM: {rouge_scores['rougeLsum']}")

# Tính BERTScore
bert_scores = bertscore.compute(predictions=predictions, references=references, lang="vi")
print("\nBERTScore:")
print(f"Precision: {sum(bert_scores['precision']) / len(bert_scores['precision']):.4f}")
print(f"Recall: {sum(bert_scores['recall']) / len(bert_scores['recall']):.4f}")
print(f"F1: {sum(bert_scores['f1']) / len(bert_scores['f1']):.4f}")

ROUGE Scores:
ROUGE-1: 0.4743536288930348
ROUGE-2: 0.23418051665741274
ROUGE-L: 0.3834867241434672
ROUGE-LSUM: 0.38344599845043165


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


BERTScore:
Precision: 0.7687
Recall: 0.7547
F1: 0.7613


In [60]:
import numpy as np
sample = np.random.randint(0, 6000)

print("- Thực tế:")
print(references[sample])
print("- Dự đoán:")
print(predictions[sample])
print("- Nội dung")
print(test_df.iloc[sample, 2])

- Thực tế:
thắp hơn 30000 ngọn nến để tưởng_nhớ các liệt_sĩ
- Dự đoán:
​ thắp nến tri_ân các anh_hùng liệt_sĩ tại bình_định
- Nội dung
tại buổi lễ các lãnh_đạo tỉnh đã thực_hiện nghi_thức thả chim bồ_câu nghi_thức này có ý_nghĩa những cánh chim bồ_câu mạnh_mẽ chở theo ước_mơ khát_vọng cống_hiến cho đất_nước của thế_hệ trẻ tỉnh bình_định dịp này ban tổ_chức đã trao_tặng 10 suất quà cho các cựu_chiến_binh cựu thanh_niên xung_phong tại tpquy nhơn cũng trong tối 267 hơn 30000 ngọn nến tại 106 nghĩa_trang trên địa_bàn tỉnh bình_định đồng_loạt được các đoàn_viên thanh_niên thắp sáng để tưởng_nhớ các liệt_sĩ đã hy_sinh vì độc_lập dân_tộc anh nguyễn_thành trung_phó bí_thư tỉnh đoàn bình_định cho biết kỷ_niệm 76 năm ngày thương_binh liệt_sĩ năm nay các cấp_bộ đoàn trong toàn tỉnh đã tổ_chức nhiều hoạt_động ý_nghĩa như hành_trình tri_ân_hành_trình đến với địa_chỉ đỏ bữa cơm yêu_thương để tỏ_lòng thành_kính và biết_ơn sâu_sắc các thế_hệ cha_anh các mẹ việt_nam anh_hùng người có công với cách_mạng

In [ ]:
# Bảng testcases
output_df = pd.DataFrame({
    "Reference": references,
    "Prediction": predictions
})

output_df['Reference'] = output_df['Reference'].str.replace('_', ' ')
output_df['Prediction'] = output_df['Prediction'].str.replace('_', ' ')

output_file_path = "/content/drive/MyDrive/data/predictions_output.csv"
output_df.to_csv(output_file_path, index=False, encoding="utf-8")

In [ ]:
# trainer.train(resume_from_checkpoint='/content/drive/MyDrive/training/checkpoint-<last_checkpoint>')